# Running inference

In [2]:
import librosa as lr
import octafilt3r.filter as o3f
import octafilt3r.plot as o3p
import os
from tensorflow.keras.models import load_model
import numpy as np
import pipeline.config as conf
import pipeline.scale as scale

In [13]:
param_dict = conf.open_params()

base_path = "Datasets/UrbanSound8k_augmented/street_music/"
stop_at = 10
i = 1
true = 0
false = 0

N4CED = load_model('N4CED_V-0-1')

for file in os.listdir(base_path):

    if(i == stop_at):
        break

    wav, fs = lr.load(base_path + file, sr=param_dict['fs'], duration=param_dict["max_dur"])

    if len(wav) < fs:
        continue


    feats, fcs = o3f.rolling_oct_bank(
        wav,
        fs,
        param_dict['oct_bw_ratio'],
        param_dict['order'],
        param_dict['fmax'],
        param_dict['fmin'],
        param_dict['frame_size'],
        param_dict['dec_stages'],
        param_dict['dec_ord']
    )

    df = scale.feats2frames(feats, param_dict['frame_size'], fs)

    X, scaler = scale.data_scaler(df)
    p = N4CED.predict(X)
    predicted = param_dict["class_map"][np.argmax(np.mean(p, axis=0))]
    print(f'Predicted class: "{predicted}" with {int(100 * (p[0][np.argmax(p, 1)][0]))} score.')
    if predicted == 'street_music':
        true += 1
    else:
        false += 1
    i += 1

print('----- done. -----')
print(f'{true} true predictions\n{false} false predictions')
print(f'test accuracy: {int((true/i)*100)}%\nerror ratio: {int((false/i)*100)}%')

Predicted class: "street_music" with 100 score.
Predicted class: "street_music" with 99 score.
Predicted class: "street_music" with 99 score.
Predicted class: "street_music" with 99 score.
Predicted class: "street_music" with 100 score.
Predicted class: "street_music" with 99 score.
Predicted class: "street_music" with 98 score.
Predicted class: "street_music" with 99 score.
Predicted class: "street_music" with 94 score.
----- done. -----
9 true predictions
0 false predictions
test accuracy: 90%
error ratio: 0%
